In [ ]:
import subprocess
import sys
import os
import sqlite3
import json
import logging
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Configure logging for console output
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(console_handler)

# Function to check and install missing packages
def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}")

# Function to upgrade pip
def upgrade_pip():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
        print("Pip upgraded successfully.")
    except subprocess.CalledProcessError:
        print("Failed to upgrade pip.")

# List of required packages
required_packages = [
    'torch', 
    'torchvision', 
    'matplotlib', 
    'numpy', 
    'scikit-learn', 
    'pandas'
]

# Upgrade pip first
upgrade_pip()

# Import required packages with error handling for missing dependencies
for package_name in required_packages:
    try:
        __import__(package_name)
        print(f"Successfully imported {package_name}")
    except ImportError:
        print(f"Package {package_name} not found. Attempting to install...")
        install_package(package_name)

# Define DatabaseManager with better connection handling
class DatabaseManager:
    def __init__(self, db_path='training_pipeline.db'):
        self.db_path = db_path
        logger.info(f"Connecting to database at {self.db_path}")
        self.conn = sqlite3.connect(db_path, timeout=5.0, check_same_thread=False)
        self.initialize_database()

    def initialize_database(self):
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS model_config (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    input_dim INTEGER,
                    hidden_dim INTEGER,
                    output_dim INTEGER,
                    dropout_prob REAL,
                    created_at TEXT NOT NULL
                )
            ''')
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS tensor_activations (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    epoch INTEGER,
                    layer TEXT,
                    activations TEXT,
                    created_at TEXT NOT NULL
                )
            ''')
            self.conn.commit()
            logger.info("Database schema initialized.")
        except sqlite3.Error as e:
            logger.error(f"Error initializing database: {e}")
            print(f"Error initializing database: {e}")

    def store_model_config(self, input_dim, hidden_dim, output_dim, dropout_prob):
        try:
            with self.conn:
                cursor = self.conn.cursor()
                cursor.execute('''
                    INSERT INTO model_config (input_dim, hidden_dim, output_dim, dropout_prob, created_at) 
                    VALUES (?, ?, ?, ?, ?)
                ''', (input_dim, hidden_dim, output_dim, dropout_prob, datetime.now().isoformat()))
            logger.info("Model configuration stored.")
        except sqlite3.IntegrityError as e:
            logger.error(f"IntegrityError: {e}")
            print(f"IntegrityError: {e}")

    def store_tensor_activations(self, epoch, layer, activations):
        try:
            with self.conn:
                cursor = self.conn.cursor()
                cursor.execute('''
                    INSERT INTO tensor_activations (epoch, layer, activations, created_at) 
                    VALUES (?, ?, ?, ?)
                ''', (epoch, layer, json.dumps(activations), datetime.now().isoformat()))
            logger.info(f"Stored activations for epoch {epoch} and layer {layer}.")
        except sqlite3.OperationalError as e:
            logger.error(f"Error storing tensor activations: {e}")
            print(f"Error storing tensor activations: {e}")

    def close(self):
        try:
            self.conn.close()
            logger.info(f"Closed database connection to {self.db_path}.")
        except sqlite3.Error as e:
            logger.error(f"Error closing database connection: {e}")
            print(f"Error closing database connection: {e}")

# Generate a synthetic dataset for binary classification
def generate_synthetic_data(samples=1000, features=20, classes=2):
    try:
        print(f"Generating synthetic dataset with {samples} samples, {features} features, and {classes} classes...")
        X, y = make_classification(n_samples=samples, n_features=features, n_classes=classes, random_state=42)
        print("Synthetic dataset generated successfully.")
        return X, y
    except Exception as e:
        print(f"Error generating synthetic dataset: {e}")
        raise

# Split and scale the dataset
def preprocess_data(X, y, test_size=0.2):
    try:
        print(f"Splitting dataset into training and testing sets with test size = {test_size}...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        
        scaler = StandardScaler()
        print("Fitting the scaler to the training data...")
        X_train = scaler.fit_transform(X_train)
        print("Transforming the test data...")
        X_test = scaler.transform(X_test)
        
        print("Data preprocessing completed successfully.")
        return X_train, X_test, y_train, y_test
    except Exception as e:
        print(f"Error during data preprocessing: {e}")
        raise

# Convert to PyTorch tensors
def convert_to_tensors(X_train, X_test, y_train, y_test):
    try:
        print("Converting training and testing data to PyTorch tensors...")
        
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)

        # Debug statements to display tensor shapes
        print(f"X_train_tensor shape: {X_train_tensor.shape}")
        print(f"X_test_tensor shape: {X_test_tensor.shape}")
        print(f"y_train_tensor shape: {y_train_tensor.shape}")
        print(f"y_test_tensor shape: {y_test_tensor.shape}")
        
        print("Conversion to PyTorch tensors completed successfully.")
        
        return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor
    
    except Exception as e:
        print(f"Error converting data to tensors: {e}")
        raise

# Function to create DataLoader objects
def create_data_loaders(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size=32):
    try:
        print("Creating DataLoader objects...")
        
        # Create TensorDataset objects
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

        # Create DataLoader objects
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Debug statements to display the number of batches
        print(f"Number of batches in training DataLoader: {len(train_loader)}")
        print(f"Number of batches in testing DataLoader: {len(test_loader)}")
        
        print("DataLoader objects created successfully.")
        
        return train_loader, test_loader
    
    except Exception as e:
        print(f"Error creating DataLoader objects: {e}")
        raise

# Call the function to create DataLoader objects
X, y = generate_synthetic_data(samples=1000, features=20, classes=2)
X_train, X_test, y_train, y_test = preprocess_data(X, y, test_size=0.2)
X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = convert_to_tensors(X_train, X_test, y_train, y_test)
train_loader, test_loader = create_data_loaders(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

# Initialize or load the database
def initialize_or_load_db(db_path='training_pipeline.db'):
    try:
        print(f"Attempting to connect to the database at {db_path}...")
        db_manager = DatabaseManager(db_path=db_path)
        print("Database connection established.")
        return db_manager
    except Exception as e:
        print(f"Error initializing or loading the database: {e}")
        raise

# Call the function to initialize or load the database
db_manager = initialize_or_load_db()

# Model Definition
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.3, db_manager=None):
        super(DNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, hidden_dim)
        self.layer4 = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

        # Store model configuration in the database
        if db_manager:
            db_manager.store_model_config(input_dim, hidden_dim, output_dim, dropout_prob)

        # Visualize initial weights
        self.visualize_weights()

    def visualize_weights(self):
        """Visualizes the weights of each layer."""
        print("Visualizing initial weights...")
        plt.figure(figsize=(12, 8))
        layers = [self.layer1, self.layer2, self.layer3, self.layer4, self.output_layer]
        for i, layer in enumerate(layers):
            weights = layer.weight.data.cpu().numpy()
            plt.subplot(3, 2, i + 1)
            plt.hist(weights.flatten(), bins=30, alpha=0.7, color='blue')
            plt.title(f'Layer {i + 1} Weights Distribution')
            plt.xlabel('Weight Value')
            plt.ylabel('Frequency')
            plt.grid()

        plt.tight_layout()
        plt.show()
        print("Initial weights visualized for all layers.")

    def forward(self, x, epoch=None, db_manager=None):
        print(f"Forward pass input shape: {x.shape}")
        
        x = self.layer1(x)
        print(f"After Layer 1: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer2(x)
        print(f"After Layer 2: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer3(x)
        print(f"After Layer 3: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer4(x)
        print(f"After Layer 4: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.output_layer(x)
        print(f"After Output Layer: {x.shape}")

        # Store activations in the database
        if db_manager and epoch is not None:
            db_manager.store_tensor_activations(epoch, "Output Layer", x.detach().cpu().numpy().tolist())

        return x

# Model instantiation and training parameters
input_dim = X_train.shape[1]  # Set input dimension based on the training data
hidden_dim = 128  # Define hidden layer dimension
output_dim = 2  # Set output dimension for binary classification
dropout_prob = 0.3  # Define dropout probability

# Initialize the DNN model
print("Initializing the DNN model...")
model = DNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_prob=dropout_prob, db_manager=db_manager)

# Set the optimizer
print("Setting up the optimizer...")
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the loss function
print("Setting up the loss function (CrossEntropyLoss)...")
criterion = nn.CrossEntropyLoss()

# Function to plot the training results
def plot_training_results(train_losses, train_accuracies, epochs):
    """
    Plots training loss and accuracy over epochs.
    
    Args:
    - train_losses (list): List of training losses.
    - train_accuracies (list): List of training accuracies.
    - epochs (int): Number of epochs.
    """
    print("Plotting training results...")
    plt.figure(figsize=(12, 5))

    # Plot training loss
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), train_losses, label='Training Loss', color='blue')
    plt.title('Training Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.xticks(range(1, epochs + 1))
    plt.legend()
    plt.grid()

    # Plot training accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), train_accuracies, label='Training Accuracy', color='green')
    plt.title('Training Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.xticks(range(1, epochs + 1))
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()
    print("Training results plotted successfully.")

# Function to train the model
def train_model(model, data_loader, optimizer, criterion, epochs=10, device='cpu', db_manager=None):
    model.to(device)
    model.train()

    train_losses = []
    train_accuracies = []

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        print(f"Starting epoch {epoch + 1}/{epochs}...")

        for batch_idx, (inputs, labels) in enumerate(data_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs, epoch=epoch, db_manager=db_manager)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(data_loader)
        accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(accuracy)

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    plot_training_results(train_losses, train_accuracies, epochs)  # Plot results after training

# Train the model
train_model(model, train_loader, optimizer, criterion, epochs=10, db_manager=db_manager)

# Close the database connection
db_manager.close()


In [ ]:
import os
import json
import sqlite3
import logging
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from datetime import datetime

# Configure logging for console output
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(console_handler)

class DatabaseManager:
    def __init__(self, db_path='training_pipeline.db'):
        self.db_path = db_path
        logger.info(f"Connecting to database at {self.db_path}")
        self.conn = sqlite3.connect(db_path, timeout=5.0, check_same_thread=False)
        self.initialize_database()

    def initialize_database(self):
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS arc_data (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                file_name TEXT NOT NULL,
                content TEXT,
                created_at TEXT NOT NULL
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS model_config (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                input_dim INTEGER,
                hidden_dim INTEGER,
                output_dim INTEGER,
                dropout_prob REAL,
                created_at TEXT NOT NULL
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS tensor_activations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                epoch INTEGER,
                layer TEXT,
                activations TEXT,
                created_at TEXT NOT NULL
            )
        ''')
        self.conn.commit()
        logger.info("Database schema initialized.")

    def store_arc_data(self, file_name, content):
        try:
            with self.conn:
                cursor = self.conn.cursor()
                cursor.execute('''
                    INSERT INTO arc_data (file_name, content, created_at) 
                    VALUES (?, ?, ?)
                ''', (file_name, content, datetime.now().isoformat()))
            logger.info(f"Stored data from {file_name}.")
        except sqlite3.IntegrityError as e:
            logger.error(f"IntegrityError: {e}")

    def store_model_config(self, input_dim, hidden_dim, output_dim, dropout_prob):
        try:
            with self.conn:
                cursor = self.conn.cursor()
                cursor.execute('''
                    INSERT INTO model_config (input_dim, hidden_dim, output_dim, dropout_prob, created_at) 
                    VALUES (?, ?, ?, ?, ?)
                ''', (input_dim, hidden_dim, output_dim, dropout_prob, datetime.now().isoformat()))
            logger.info("Model configuration stored.")
        except sqlite3.IntegrityError as e:
            logger.error(f"IntegrityError: {e}")

    def store_tensor_activations(self, epoch, layer, activations):
        try:
            with self.conn:
                cursor = self.conn.cursor()
                cursor.execute('''
                    INSERT INTO tensor_activations (epoch, layer, activations, created_at) 
                    VALUES (?, ?, ?, ?)
                ''', (epoch, layer, json.dumps(activations), datetime.now().isoformat()))
            logger.info(f"Stored activations for epoch {epoch} and layer {layer}.")
        except sqlite3.OperationalError as e:
            logger.error(f"Error storing tensor activations: {e}")

    def close(self):
        try:
            self.conn.close()
            logger.info(f"Closed database connection to {self.db_path}.")
        except sqlite3.Error as e:
            logger.error(f"Error closing database connection: {e}")

def search_and_document_arc_data(folder_path):
    db_manager = DatabaseManager()

    try:
        print(f"Searching for JSON files in {folder_path}...")
        for filename in os.listdir(folder_path):
            if filename.endswith('.json'):
                file_path = os.path.join(folder_path, filename)
                print(f"Processing file: {file_path}")
                
                with open(file_path, 'r') as f:
                    content = f.read()
                    db_manager.store_arc_data(filename, content)
    
    except Exception as e:
        print(f"Error during searching or processing: {e}")
    
    finally:
        db_manager.close()

# Generate a synthetic dataset for binary classification
def generate_synthetic_data(samples=1000, features=20, classes=2):
    try:
        print(f"Generating synthetic dataset with {samples} samples, {features} features, and {classes} classes...")
        X, y = make_classification(n_samples=samples, n_features=features, n_classes=classes, random_state=42)
        print("Synthetic dataset generated successfully.")
        return X, y
    except Exception as e:
        print(f"Error generating synthetic dataset: {e}")
        raise

# Split and scale the dataset
def preprocess_data(X, y, test_size=0.2):
    try:
        print(f"Splitting dataset into training and testing sets with test size = {test_size}...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        
        scaler = StandardScaler()
        print("Fitting the scaler to the training data...")
        X_train = scaler.fit_transform(X_train)
        print("Transforming the test data...")
        X_test = scaler.transform(X_test)
        
        print("Data preprocessing completed successfully.")
        return X_train, X_test, y_train, y_test
    except Exception as e:
        print(f"Error during data preprocessing: {e}")
        raise

# Convert to PyTorch tensors
def convert_to_tensors(X_train, X_test, y_train, y_test):
    try:
        print("Converting training and testing data to PyTorch tensors...")
        
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)

        print(f"X_train_tensor shape: {X_train_tensor.shape}")
        print(f"X_test_tensor shape: {X_test_tensor.shape}")
        print(f"y_train_tensor shape: {y_train_tensor.shape}")
        print(f"y_test_tensor shape: {y_test_tensor.shape}")
        
        print("Conversion to PyTorch tensors completed successfully.")
        
        return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor
    
    except Exception as e:
        print(f"Error converting data to tensors: {e}")
        raise

# Function to create DataLoader objects
def create_data_loaders(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size=32):
    try:
        print("Creating DataLoader objects...")
        
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        print(f"Number of batches in training DataLoader: {len(train_loader)}")
        print(f"Number of batches in testing DataLoader: {len(test_loader)}")
        
        print("DataLoader objects created successfully.")
        
        return train_loader, test_loader
    
    except Exception as e:
        print(f"Error creating DataLoader objects: {e}")
        raise

# Model Definition
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.3, db_manager=None):
        super(DNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, hidden_dim)
        self.layer4 = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

        if db_manager:
            db_manager.store_model_config(input_dim, hidden_dim, output_dim, dropout_prob)

        self.visualize_weights()

    def visualize_weights(self):
        print("Visualizing initial weights...")
        plt.figure(figsize=(12, 8))
        layers = [self.layer1, self.layer2, self.layer3, self.layer4, self.output_layer]
        for i, layer in enumerate(layers):
            weights = layer.weight.data.cpu().numpy()
            plt.subplot(3, 2, i + 1)
            plt.hist(weights.flatten(), bins=30, alpha=0.7, color='blue')
            plt.title(f'Layer {i + 1} Weights Distribution')
            plt.xlabel('Weight Value')
            plt.ylabel('Frequency')
            plt.grid()

        plt.tight_layout()
        plt.show()
        print("Initial weights visualized for all layers.")

    def forward(self, x, epoch=None, db_manager=None):
        print(f"Forward pass input shape: {x.shape}")
        
        x = self.layer1(x)
        print(f"After Layer 1: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer2(x)
        print(f"After Layer 2: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer3(x)
        print(f"After Layer 3: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer4(x)
        print(f"After Layer 4: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.output_layer(x)
        print(f"After Output Layer: {x.shape}")

        if db_manager and epoch is not None:
            db_manager.store_tensor_activations(epoch, "Output Layer", x.detach().cpu().numpy().tolist())

        return x

# Function to plot training results
def plot_training_results(train_losses, train_accuracies, epochs):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), train_losses, label='Training Loss', color='blue')
    plt.title('Training Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.xticks(range(1, epochs + 1))
    plt.legend()
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), train_accuracies, label='Training Accuracy', color='green')
    plt.title('Training Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.xticks(range(1, epochs + 1))
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()
    print("Training results plotted successfully.")

# Function to train the model
def train_model(model, data_loader, optimizer, criterion, epochs=10, device='cpu', db_manager=None):
    model.to(device)
    model.train()

    train_losses = []
    train_accuracies = []

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        print(f"Starting epoch {epoch + 1}/{epochs}...")
        logger.info(f"Starting epoch {epoch + 1}/{epochs}...")

        for batch_idx, (inputs, labels) in enumerate(data_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs, epoch=epoch, db_manager=db_manager)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(data_loader)
        accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(accuracy)

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")
        logger.info(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    plot_training_results(train_losses, train_accuracies, epochs)

# Main execution
folder_path = r"C:\Users\Owner\Downloads\arc-prize-2024"
search_and_document_arc_data(folder_path)

# Generate synthetic dataset for binary classification
X, y = generate_synthetic_data(samples=1000, features=20, classes=2)
X_train, X_test, y_train, y_test = preprocess_data(X, y, test_size=0.2)

# Convert to PyTorch tensors
X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = convert_to_tensors(X_train, X_test, y_train, y_test)

# Create DataLoader objects
train_loader, test_loader = create_data_loaders(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

# Initialize or load the database
db_manager = initialize_or_load_db()

# Model instantiation and training parameters
input_dim = X_train.shape[1]  # Set input dimension based on the training data
hidden_dim = 128  # Define hidden layer dimension
output_dim = 2  # Set output dimension for binary classification
dropout_prob = 0.3  # Define dropout probability

# Initialize the DNN model
print("Initializing the DNN model...")
model = DNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_prob=dropout_prob, db_manager=db_manager)

# Set the optimizer
print(f"Setting up the optimizer with learning rate = 0.001...")
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("Optimizer set up successfully.")

# Set the loss function
print("Setting up the loss function (CrossEntropyLoss)...")
criterion = nn.CrossEntropyLoss()
print("Loss function set up successfully.")

# Train the model
train_model(model, train_loader, optimizer, criterion, epochs=10, db_manager=db_manager)

# Close the database connection
db_manager.close()


In [ ]:
import subprocess
import sys
import os
import sqlite3
import json
import logging
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Configure logging for console output
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(console_handler)

# Function to check and install missing packages
def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}")

# Function to upgrade pip
def upgrade_pip():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
        print("Pip upgraded successfully.")
    except subprocess.CalledProcessError:
        print("Failed to upgrade pip.")

# List of required packages
required_packages = [
    'torch', 
    'torchvision', 
    'matplotlib', 
    'numpy', 
    'scikit-learn', 
    'pandas'
]

# Upgrade pip first
upgrade_pip()

# Import required packages with error handling for missing dependencies
for package_name in required_packages:
    try:
        __import__(package_name)
        print(f"Successfully imported {package_name}")
    except ImportError:
        print(f"Package {package_name} not found. Attempting to install...")
        install_package(package_name)

# Define DatabaseManager with better connection handling
class DatabaseManager:
    def __init__(self, db_path='training_pipeline.db'):
        self.db_path = db_path
        logger.info(f"Connecting to database at {self.db_path}")
        self.conn = sqlite3.connect(db_path, timeout=5.0, check_same_thread=False)
        self.initialize_database()

    def initialize_database(self):
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS model_config (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                input_dim INTEGER,
                hidden_dim INTEGER,
                output_dim INTEGER,
                dropout_prob REAL,
                created_at TEXT NOT NULL
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS tensor_activations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                epoch INTEGER,
                layer TEXT,
                activations TEXT,
                created_at TEXT NOT NULL
            )
        ''')
        self.conn.commit()
        logger.info("Database schema initialized.")

    def store_model_config(self, input_dim, hidden_dim, output_dim, dropout_prob):
        try:
            with self.conn:
                cursor = self.conn.cursor()
                cursor.execute('''
                    INSERT INTO model_config (input_dim, hidden_dim, output_dim, dropout_prob, created_at) 
                    VALUES (?, ?, ?, ?, ?)
                ''', (input_dim, hidden_dim, output_dim, dropout_prob, datetime.now().isoformat()))
            logger.info("Model configuration stored.")
        except sqlite3.IntegrityError as e:
            logger.error(f"IntegrityError: {e}")

    def store_tensor_activations(self, epoch, layer, activations):
        try:
            with self.conn:
                cursor = self.conn.cursor()
                cursor.execute('''
                    INSERT INTO tensor_activations (epoch, layer, activations, created_at) 
                    VALUES (?, ?, ?, ?)
                ''', (epoch, layer, json.dumps(activations), datetime.now().isoformat()))
            logger.info(f"Stored activations for epoch {epoch} and layer {layer}.")
        except sqlite3.OperationalError as e:
            logger.error(f"Error storing tensor activations: {e}")

    def close(self):
        try:
            self.conn.close()
            logger.info(f"Closed database connection to {self.db_path}.")
        except sqlite3.Error as e:
            logger.error(f"Error closing database connection: {e}")

# Generate a synthetic dataset for classification
def generate_synthetic_data(samples=1000, features=20, classes=5):
    try:
        print(f"Generating synthetic dataset with {samples} samples, {features} features, and {classes} classes...")
        # Adjusting the number of informative features to avoid ValueError
        n_informative = min(features, classes - 1)  # Ensure we do not exceed limits
        X, y = make_classification(n_samples=samples, n_features=features, n_classes=classes, 
                                   n_informative=n_informative, n_clusters_per_class=1, random_state=42)
        print("Synthetic dataset generated successfully.")
        return X, y
    except Exception as e:
        print(f"Error generating synthetic dataset: {e}")
        raise

# Split and scale the dataset
def preprocess_data(X, y, test_size=0.2):
    try:
        print(f"Splitting dataset into training and testing sets with test size = {test_size}...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        
        scaler = StandardScaler()
        print("Fitting the scaler to the training data...")
        X_train = scaler.fit_transform(X_train)
        print("Transforming the test data...")
        X_test = scaler.transform(X_test)
        
        print("Data preprocessing completed successfully.")
        return X_train, X_test, y_train, y_test
    except Exception as e:
        print(f"Error during data preprocessing: {e}")
        raise

# Convert to PyTorch tensors
def convert_to_tensors(X_train, X_test, y_train, y_test):
    try:
        print("Converting training and testing data to PyTorch tensors...")
        
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)

        # Debug statements to display tensor shapes
        print(f"X_train_tensor shape: {X_train_tensor.shape}")
        print(f"X_test_tensor shape: {X_test_tensor.shape}")
        print(f"y_train_tensor shape: {y_train_tensor.shape}")
        print(f"y_test_tensor shape: {y_test_tensor.shape}")
        
        print("Conversion to PyTorch tensors completed successfully.")
        
        return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor
    
    except Exception as e:
        print(f"Error converting data to tensors: {e}")
        raise

# Function to create DataLoader objects
def create_data_loaders(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size=32):
    try:
        print("Creating DataLoader objects...")
        
        # Create TensorDataset objects
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

        # Create DataLoader objects
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Debug statements to display the number of batches
        print(f"Number of batches in training DataLoader: {len(train_loader)}")
        print(f"Number of batches in testing DataLoader: {len(test_loader)}")
        
        print("DataLoader objects created successfully.")
        
        return train_loader, test_loader
    
    except Exception as e:
        print(f"Error creating DataLoader objects: {e}")
        raise

# Model Definition
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.3, db_manager=None):
        super(DNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, hidden_dim)
        self.layer4 = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

        # Store model configuration in the database
        if db_manager:
            db_manager.store_model_config(input_dim, hidden_dim, output_dim, dropout_prob)

        # Visualize initial weights
        self.visualize_weights()

    def visualize_weights(self):
        """Visualizes the weights of each layer."""
        print("Visualizing initial weights...")
        plt.figure(figsize=(12, 8))
        layers = [self.layer1, self.layer2, self.layer3, self.layer4, self.output_layer]
        for i, layer in enumerate(layers):
            weights = layer.weight.data.cpu().numpy()
            plt.subplot(3, 2, i + 1)
            plt.hist(weights.flatten(), bins=30, alpha=0.7, color='blue')
            plt.title(f'Layer {i + 1} Weights Distribution')
            plt.xlabel('Weight Value')
            plt.ylabel('Frequency')
            plt.grid()

        plt.tight_layout()
        plt.show()
        print("Initial weights visualized for all layers.")

    def forward(self, x, epoch=None, db_manager=None):
        print(f"Forward pass input shape: {x.shape}")
        
        x = self.layer1(x)
        print(f"After Layer 1: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer2(x)
        print(f"After Layer 2: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer3(x)
        print(f"After Layer 3: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer4(x)
        print(f"After Layer 4: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.output_layer(x)
        print(f"After Output Layer: {x.shape}")

        # Store activations in the database
        if db_manager and epoch is not None:
            db_manager.store_tensor_activations(epoch, "Output Layer", x.detach().cpu().numpy().tolist())

        return x

# Function to train the model
def train_model(model, data_loader, optimizer, criterion, epochs=10, device='cpu', db_manager=None):
    model.to(device)
    model.train()

    train_losses = []
    train_accuracies = []

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        print(f"Starting epoch {epoch + 1}/{epochs}...")
        logger.info(f"Starting epoch {epoch + 1}/{epochs}...")

        for batch_idx, (inputs, labels) in enumerate(data_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs, epoch=epoch, db_manager=db_manager)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(data_loader)
        accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(accuracy)

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")
        logger.info(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    return train_losses, train_accuracies  # Return the losses and accuracies for plotting

# 3D Plotting function
def plot_3d_training_results(train_losses, train_accuracies, epochs):
    print("Plotting 3D training results...")
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    x = np.arange(1, epochs + 1)

    for i in range(len(train_losses)):
        ax.plot(x, train_losses[i], train_accuracies[i], label=f'Model {i + 1}', marker='o')

    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.set_zlabel('Accuracy (%)')
    ax.set_title('Training Loss and Accuracy over Epochs')
    ax.legend()
    plt.show()

# Main execution
input_dim = 20  # Example input dimension
hidden_dim = 128  # Example hidden layer dimension
output_dim = 5  # Example output dimension for 5 classes
dropout_prob = 0.3  # Define dropout probability
epochs = 100  # Define number of epochs

# Prepare to store results for all models
all_train_losses = []
all_train_accuracies = []

# Train multiple models
num_models = 10
for model_index in range(num_models):
    # Generate synthetic dataset for classification
    X, y = generate_synthetic_data(samples=1000, features=input_dim, classes=5)
    X_train, X_test, y_train, y_test = preprocess_data(X, y, test_size=0.2)
    X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = convert_to_tensors(X_train, X_test, y_train, y_test)

    # Create data loaders
    train_loader, test_loader = create_data_loaders(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

    # Initialize database
    db_manager = DatabaseManager()

    # Initialize the DNN model
    print(f"Initializing model {model_index + 1}...")
    model = DNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_prob=dropout_prob, db_manager=db_manager)

    # Set the optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Set the loss function
    criterion = nn.CrossEntropyLoss()

    # Train the model and store results
    train_losses, train_accuracies = train_model(model, train_loader, optimizer, criterion, epochs=epochs, db_manager=db_manager)
    all_train_losses.append(train_losses)
    all_train_accuracies.append(train_accuracies)

    # Close the database connection
    db_manager.close()

# Plotting the results for all models
plot_3d_training_results(all_train_losses, all_train_accuracies, epochs)



In [ ]:
import subprocess
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
import json
import time
from tqdm import tqdm

# Function to check and install missing packages
def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}")

# List of required packages
required_packages = [
    'torch',
    'numpy',
    'pandas',
    'matplotlib',
    'tqdm',
]

# Import required packages with error handling for missing dependencies
for package_name in required_packages:
    try:
        __import__(package_name)
        print(f"Successfully imported {package_name}")
    except ImportError:
        print(f"Package {package_name} not found. Attempting to install...")
        install_package(package_name)

In [ ]:
# Set device for PyTorch
try:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
except Exception as e:
    print(f"Error setting device: {e}")

# Set up plotting parameters
try:
    plt.rcParams["font.family"] = "serif"
    print("Plotting parameters set successfully.")
except Exception as e:
    print(f"Error setting plotting parameters: {e}")

In [ ]:
import torch
import matplotlib.pyplot as plt

# Padding function for input grids
def pad_to_30x30(grid, pad_value=10):
    """Pads the grid to a size of 30x30 with the specified pad_value."""
    try:
        # Create a padded grid filled with the pad_value
        padded_grid = torch.full((30, 30), pad_value, dtype=torch.int8)
        
        # Determine the height and width of the input grid
        height, width = len(grid), len(grid[0])
        
        # Fill the padded grid with the input grid values
        padded_grid[:height, :width] = torch.tensor(grid, dtype=torch.int8)
        
        print(f"Padded grid created successfully with shape: {padded_grid.shape}")
        return padded_grid
    except Exception as e:
        print(f"Error in padding the grid: {e}")

def visualize_padded_grid(grid):
    """Visualizes the padded grid."""
    try:
        padded_grid = pad_to_30x30(grid)  # Get the padded grid
        plt.imshow(padded_grid, cmap='gray', interpolation='nearest')  # Plot the grid
        plt.colorbar()  # Add a colorbar for reference
        plt.title("Padded Grid Visualization")  # Title for the plot
        plt.xlabel("Width (Columns)")  # X-axis label
        plt.ylabel("Height (Rows)")  # Y-axis label
        plt.show()  # Display the plot
        print("Padded grid visualized successfully.")
    except Exception as e:
        print(f"Error visualizing padded grid: {e}")

# Example usage of the padding and visualization
if __name__ == "__main__":
    # Define a sample grid (2D list)
    sample_grid = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

    print("Original Grid:")
    print(torch.tensor(sample_grid))  # Print the original grid

    # Visualize the padded grid
    visualize_padded_grid(sample_grid)


In [ ]:
def preprocess_images(dataset):
    """Preprocesses the dataset by converting images to one-hot encoding."""
    try:
        one_hot_images = F.one_hot(dataset, num_classes=11)  # Assuming 11 classes
        one_hot_images = one_hot_images.permute(0, 3, 1, 2)  # Rearrange dimensions for PyTorch
        print(f"Preprocessed images to one-hot encoding with shape: {one_hot_images.shape}")
        return one_hot_images.float()
    except Exception as e:
        print(f"Error in preprocessing images: {e}")

def visualize_one_hot_images(one_hot_images):
    """Visualizes one-hot encoded images."""
    try:
        # Assuming the one-hot images are of shape (N, C, H, W)
        num_images = one_hot_images.shape[0]
        fig, axs = plt.subplots(1, num_images, figsize=(15, 5))
        
        for i in range(num_images):
            axs[i].imshow(one_hot_images[i].permute(1, 2, 0).numpy())  # Convert back to HWC for plotting
            axs[i].axis('off')
            axs[i].set_title(f'Image {i + 1}')
        
        plt.suptitle("One-Hot Encoded Images Visualization")
        plt.show()
        print("One-hot encoded images visualized successfully.")
    except Exception as e:
        print(f"Error visualizing one-hot images: {e}")

def preprocess_text(text_data):
    """Preprocesses text data by tokenization and lowercasing."""
    try:
        preprocessed_text = [text.lower() for text in text_data]  # Convert to lowercase
        print(f"Preprocessed text: {preprocessed_text}")
        return preprocessed_text
    except Exception as e:
        print(f"Error in preprocessing text: {e}")

def preprocess_audio(audio_data):
    """Preprocesses audio data (placeholder function)."""
    try:
        # Implement audio preprocessing logic, e.g., normalization, filtering, etc.
        print(f"Audio data processed: {audio_data}")  # Placeholder
        return audio_data  # Return processed audio
    except Exception as e:
        print(f"Error in preprocessing audio: {e}")

# Example usage of preprocessing functions
if __name__ == "__main__":
    # Define a sample dataset (tensor of image classes)
    sample_images = torch.randint(0, 11, (5, 30, 30))  # 5 images with random class labels
    print("Original Image Data:")
    print(sample_images)

    # Preprocess and visualize images
    one_hot_encoded_images = preprocess_images(sample_images)
    visualize_one_hot_images(one_hot_encoded_images)

    # Define sample text data
    sample_text = ["Hello World!", "Preprocessing is fun."]
    print("Original Text Data:")
    print(sample_text)
    
    # Preprocess text
    preprocessed_text = preprocess_text(sample_text)

    # Define sample audio data (placeholder)
    sample_audio = [0.5, -0.1, 0.7]  # Placeholder audio data
    print("Original Audio Data:")
    print(sample_audio)

    # Preprocess audio
    preprocessed_audio = preprocess_audio(sample_audio)



In [ ]:
def count_parameters(model):
    """Counts the number of trainable parameters in a model."""
    try:
        num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Total trainable parameters: {num_params}")
        return num_params
    except Exception as e:
        print(f"Error counting parameters: {e}")

# Set up color map for visualizations
def setup_colormap():
    """Sets up the colormap for visualizations."""
    try:
        _cmap = colors.ListedColormap(
            ['#000000', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
             '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25', "#FFFFFF"]
        )
        _norm = colors.Normalize(vmin=0, vmax=10)
        print("Colormap and normalization set up successfully.")
        return _cmap, _norm
    except Exception as e:
        print(f"Error setting up colormap: {e}")
        return None, None  # Return None in case of error

# Example usage
if __name__ == "__main__":
    # Create a simple model for demonstration
    class SimpleModel(nn.Module):
        def __init__(self):
            super(SimpleModel, self).__init__()
            self.fc1 = nn.Linear(10, 5)
            self.fc2 = nn.Linear(5, 2)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return x

    # Instantiate the model
    model = SimpleModel()
    print("Model Architecture:")
    print(model)

    # Count parameters
    count_parameters(model)

    # Set up colormap
    colormap, normalization = setup_colormap()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors

# Setup colormap and normalization (assuming this is defined somewhere in your code)
_cmap = colors.ListedColormap(
    ['#000000', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
     '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25', "#FFFFFF"]
)
_norm = colors.Normalize(vmin=0, vmax=10)

def plot_img(input_matrix, ax, title=""):
    """Plots an input matrix as an image."""
    try:
        ax.imshow(input_matrix, cmap=_cmap, norm=_norm)
        ax.grid(True, which='both', color='lightgrey', linewidth=0.5)
        ax.set_yticks([x - 0.5 for x in range(1 + len(input_matrix))])
        ax.set_xticks([x - 0.5 for x in range(1 + len(input_matrix[0]))]) 
        
        if title:
            ax.set_title(title)
        ax.set_xticklabels([])
        ax.set_yticklabels([])

        print(f"Successfully plotted image with title: '{title}'")
    except Exception as e:
        print(f"Error plotting image: {e}")

# Example usage
if __name__ == "__main__":
    # Create a figure and axes
    fig, ax = plt.subplots(figsize=(5, 5))
    
    # Generate a sample input matrix (e.g., 10x10 grid with random values)
    sample_matrix = np.random.randint(0, 10, size=(10, 10))

    # Plot the sample matrix
    plot_img(sample_matrix, ax, title="Sample Image")
    
    # Show the plot
    plt.show()



In [ ]:
import os

# Set device for PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set up plotting parameters
plt.rcParams["font.family"] = "serif"

def search_file(filename, search_path):
    """Search for a file in the specified directory."""
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            print(f"Found file: {os.path.join(root, filename)}")
            return os.path.join(root, filename)
    print(f"File not found: {filename}")
    return None

# Define paths for datasets
train_challenges_filename = "arc-agi_training_challenges.json"
train_solutions_filename = "arc-agi_training_solutions.json"

# Load the dataset
print("Loading the training challenges dataset...")
train_challenges_path = search_file(train_challenges_filename, ".")
train_solutions_path = search_file(train_solutions_filename, ".")

if train_challenges_path is not None and train_solutions_path is not None:
    try:
        with open(train_challenges_path, 'r') as f:
            train_challenges = json.load(f)
            print("Training challenges dataset loaded successfully.")
    except Exception as e:
        print(f"Error loading training challenges dataset: {e}")
        exit()

    try:
        with open(train_solutions_path, 'r') as f:
            train_solutions = json.load(f)
            print("Training solutions dataset loaded successfully.")
    except Exception as e:
        print(f"Error loading training solutions dataset: {e}")
        exit()

    train_ids = list(train_challenges.keys())
else:
    print("Failed to load datasets. Exiting.")
    exit()

In [ ]:
import torch
import torch.nn as nn

# Neural Network Components
class ResBlock(nn.Module):
    def __init__(self, C: int, dropout_prob: float):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.bnorm1 = nn.BatchNorm2d(C)
        self.bnorm2 = nn.BatchNorm2d(C)
        self.conv1 = nn.Conv2d(C, C, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(C, C, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(p=dropout_prob)
        
        print(f"Initialized ResBlock with {C} channels and {dropout_prob} dropout probability.")

    def forward(self, x):
        try:
            r = self.conv1(self.relu(self.bnorm1(x)))
            r = self.dropout(r)
            r = self.conv2(self.relu(self.bnorm2(r)))
            print("Forward pass successful in ResBlock.")
            return r + x
        except Exception as e:
            print(f"Error during forward pass in ResBlock: {e}")
            return None


In [ ]:
import torch
import torch.nn as nn

class ConvBlock(nn.Module):
    def __init__(self, mode: str, C_in: int, C_out: int, dropout_prob: float):
        super().__init__()
        self.relu = nn.ReLU()
        self.bnorm = nn.BatchNorm2d(C_out)
        
        # Initialize the convolution layer based on the mode
        if mode == "down":
            self.conv = nn.Conv2d(C_in, C_out, kernel_size=4, stride=2, padding=0)
            print(f"Initialized ConvBlock in 'down' mode with {C_in} input channels and {C_out} output channels.")
        elif mode == "up":
            self.conv = nn.ConvTranspose2d(C_in, C_out, kernel_size=4, stride=2, padding=0)
            print(f"Initialized ConvBlock in 'up' mode with {C_in} input channels and {C_out} output channels.")
        elif mode == "same":
            self.conv = nn.Conv2d(C_in, C_out, kernel_size=3, padding=1)
            print(f"Initialized ConvBlock in 'same' mode with {C_in} input channels and {C_out} output channels.")
        else:
            raise ValueError("Wrong ConvBlock mode.")

        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, z):
        try:
            x = self.conv(z)
            x = self.bnorm(x)
            x = self.relu(x)
            x = self.dropout(x)
            print("Forward pass successful in ConvBlock.")
            return x
        except Exception as e:
            print(f"Error during forward pass in ConvBlock: {e}")
            return None


In [ ]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, channels=[256, 512, 512], latent_dim=512, dropout=0.1):
        super(Encoder, self).__init__()
        
        # Initialize convolutional and residual blocks
        try:
            self.conv1 = ConvBlock("down", 11, channels[0], dropout)
            self.res12 = ResBlock(channels[0], dropout)
            self.conv2 = ConvBlock("down", channels[0], channels[1], dropout)
            self.res23 = ResBlock(channels[1], dropout)
            self.conv3 = ConvBlock("down", channels[1], channels[2], dropout)
            self.fc = nn.Linear(channels[2] * 2 * 2, latent_dim)
            print(f"Encoder initialized with channels: {channels}, latent_dim: {latent_dim}, dropout: {dropout}.")
        except Exception as e:
            print(f"Error during Encoder initialization: {e}")

    def forward(self, z):
        residuals = [0] * 3
        try:
            x = preprocess_images(z)  # Preprocess the input
            x = self.conv1(x)  # Apply the first convolution
            x = self.res12(x)  # Apply the first residual block
            residuals[0] = x  # Store the residual
            x = self.conv2(x)  # Apply the second convolution
            x = self.res23(x)  # Apply the second residual block
            residuals[1] = x  # Store the residual
            x = self.conv3(x)  # Apply the third convolution
            residuals[2] = x  # Store the residual
            
            x = x.reshape(x.size(0), -1)  # Flatten the output
            encoded = self.fc(x)  # Linear transformation
            print("Forward pass successful in Encoder.")
            return encoded, residuals
        except Exception as e:
            print(f"Error during forward pass in Encoder: {e}")
            return None, None


In [ ]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, channels=[256, 512, 512], latent_dim=512, dropout=0.1):
        super(Decoder, self).__init__()
        self.channels = channels
        
        try:
            self.fc = nn.Linear(latent_dim, channels[-1] * 2 * 2)
            self.conv3 = ConvBlock("up", channels[-1] * 2, channels[-2], dropout)
            self.res32 = ResBlock(channels[-2], dropout)
            self.conv2 = ConvBlock("up", channels[-2] * 2, channels[-3], dropout)
            self.res21 = ResBlock(channels[-3], dropout)
            self.conv1 = ConvBlock("up", channels[-3] * 2, channels[-3], dropout)
            self.conv0 = nn.Conv2d(channels[-3], 11, kernel_size=3, padding=1)
            print(f"Decoder initialized with channels: {channels}, latent_dim: {latent_dim}, dropout: {dropout}.")
        except Exception as e:
            print(f"Error during Decoder initialization: {e}")

    def forward(self, z, residuals):
        try:
            x = self.fc(z)  # Apply the fully connected layer
            x = x.reshape(x.size(0), self.channels[-1], 2, 2)  # Reshape for convolutional layers
            
            # Concatenate and apply the decoding layers
            x = torch.cat((x, residuals[2]), dim=1)
            x = self.conv3(x)
            x = self.res32(x)
            x = torch.cat((x, residuals[1]), dim=1)
            x = self.conv2(x)
            x = self.res21(x)
            x = torch.cat((x, residuals[0]), dim=1)
            x = self.conv1(x)
            x = self.conv0(x)  # Final convolution
            
            print("Forward pass successful in Decoder.")
            return x
        except Exception as e:
            print(f"Error during forward pass in Decoder: {e}")
            return None


In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1):
        super(MLP, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        try:
            self.fc1 = nn.Linear(input_size, hidden_size)
            self.bn1 = nn.BatchNorm1d(hidden_size)
            self.fc2 = nn.Linear(hidden_size, hidden_size)
            self.bn2 = nn.BatchNorm1d(hidden_size)
            self.fc3 = nn.Linear(hidden_size, output_size)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(p=dropout)
            print(f"MLP initialized with input_size: {input_size}, hidden_size: {hidden_size}, output_size: {output_size}, dropout: {dropout}.")
        except Exception as e:
            print(f"Error during MLP initialization: {e}")

    def forward(self, z):
        try:
            x = self.relu(self.bn1(self.fc1(z)))  # First layer with batch normalization and ReLU
            x = self.dropout(x)  # Apply dropout
            x = self.relu(self.bn2(self.fc2(x)))  # Second layer with batch normalization and ReLU
            x = self.dropout(x)  # Apply dropout
            output = self.fc3(x)  # Final output layer
            
            print("Forward pass successful in MLP.")
            return output
        except Exception as e:
            print(f"Error during forward pass in MLP: {e}")
            return None


In [ ]:
import torch
import torch.nn as nn

class Magician(nn.Module):
    def __init__(self, channels, latent_dim, hidden_size, dropout=0.1):
        super(Magician, self).__init__()
        self.channels = channels
        self.latent_dim = latent_dim
        self.hidden_size = hidden_size
        
        try:
            self.encoder = Encoder(channels, latent_dim, dropout)
            self.decoder = Decoder(channels, latent_dim, dropout)
            self.mlp_key = MLP(latent_dim * 6, hidden_size, latent_dim)
            self.mlp_map = MLP(latent_dim * 2, hidden_size, latent_dim)
            print(f"Magician initialized with channels: {channels}, latent_dim: {latent_dim}, hidden_size: {hidden_size}, dropout: {dropout}.")
        except Exception as e:
            print(f"Error during Magician initialization: {e}")

    def forward(self, input, edu_pairs):
        try:
            edu_pairs_encoded = edu_pairs.flatten(end_dim=-3)
            edu_pairs_encoded, _ = self.encoder(edu_pairs_encoded)
            edu_pairs_encoded = edu_pairs_encoded.reshape(2, -1, 3, self.latent_dim).permute(1, 0, 2, 3).flatten(start_dim=1)
            key = self.mlp_key(edu_pairs_encoded)

            input_encoded, residuals = self.encoder(input)
            output_encoded = self.mlp_map(torch.cat((input_encoded, key), dim=1))
            output_decoded = self.decoder(output_encoded, residuals)
            
            print("Forward pass successful in Magician.")
            return output_decoded
        except Exception as e:
            print(f"Error during forward pass in Magician: {e}")
            return None


In [ ]:
def model_pred(id_val, model):
    try:
        # Load educational examples for the specified task ID
        edu_examples = train_challenges[id_val]["train"]
        edu_examples = edu_examples[:3]

        # Ensure there are exactly 3 examples
        while len(edu_examples) < 3:
            edu_examples += [edu_examples[-1]]

        # Pad the input grid to 30x30
        input = pad_to_30x30(train_challenges[id_val]["test"][0]["input"]).unsqueeze(0)
        edu_pairs = torch.zeros((2, 1, 3, 30, 30), dtype=torch.int64)

        for j in range(3):
            edu_pairs[0, 0, j] = pad_to_30x30(edu_examples[j]["input"])
            edu_pairs[1, 0, j] = pad_to_30x30(edu_examples[j]["output"])

        # Convert to long type
        input = input.long()
        edu_pairs = edu_pairs.long()

        # Make predictions using the model
        output_pred_padded = torch.argmax(model(input, edu_pairs)[0], dim=0)

        # Calculate the limits for horizontal and vertical dimensions
        lim_hor = (output_pred_padded[0] < 10).sum()
        lim_ver = (output_pred_padded[:, 0] < 10).sum()

        # Slice the output prediction based on calculated limits
        output_pred = output_pred_padded[:lim_ver, :lim_hor]

        print(f"Predictions generated successfully for task ID: {id_val}.")
        return output_pred
    except Exception as e:
        print(f"Error during prediction for task ID {id_val}: {e}")
        return None


In [ ]:
def show_task_prediction(id_val, model):
    try:
        # Load the true output for the specified task ID
        output_true = np.array(train_solutions[id_val][0])
        
        # Generate predictions using the model
        output_pred = model_pred(id_val, model).cpu().numpy()
        
        # Check if the shapes of predicted and true outputs match
        if output_pred is None:
            print(f"Prediction failed for task ID {id_val}. Skipping display.")
            return
        
        if output_pred.shape == output_true.shape:
            mes = f"#err = {(output_pred != output_true).sum()}"
        else:
            mes = f"wrong shape: predicted shape {output_pred.shape}, expected shape {output_true.shape}"

        # Load educational examples for visualization
        imgs = train_challenges[id_val]["train"]
        edu_examples_num = min(len(imgs), 3)

        # Create subplots for visualization
        fig, axs = plt.subplots(2, edu_examples_num + 2, dpi=150, figsize=((edu_examples_num + 2) * 3, 6))

        # Plot educational inputs and outputs
        for j in range(edu_examples_num):
            plot_img(imgs[j]["input"], axs[0, j], "edu input")
            plot_img(imgs[j]["output"], axs[1, j], "edu output")
        
        # Plot the test input, true output, and predicted output
        plot_img(train_challenges[id_val]["test"][0]["input"], axs[0, edu_examples_num], "input")
        plot_img(output_true, axs[1, edu_examples_num + 1], "output true")
        plot_img(output_pred, axs[1, edu_examples_num], "output pred")

        # Remove the unused subplot
        fig.delaxes(axs[0, edu_examples_num + 1])
        
        # Set the title for the figure
        plt.suptitle(f"{id_val}: {mes}")
        
        # Show the plots
        plt.show()
        
        print(f"Displayed predictions for task ID: {id_val}.")
    except Exception as e:
        print(f"Error displaying prediction for task ID {id_val}: {e}")


In [ ]:
# Generate a synthetic dataset for binary classification
def generate_synthetic_data(samples=1000, features=20, classes=2):
    try:
        print(f"Generating synthetic dataset with {samples} samples, {features} features, and {classes} classes...")
        X, y = make_classification(n_samples=samples, n_features=features, n_classes=classes, random_state=42)
        print("Synthetic dataset generated successfully.")
        return X, y
    except Exception as e:
        print(f"Error generating synthetic dataset: {e}")
        raise

# Split and scale the dataset
def preprocess_data(X, y, test_size=0.2):
    try:
        print(f"Splitting dataset into training and testing sets with test size = {test_size}...")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        
        scaler = StandardScaler()
        print("Fitting the scaler to the training data...")
        X_train = scaler.fit_transform(X_train)
        print("Transforming the test data...")
        X_test = scaler.transform(X_test)
        
        print("Data preprocessing completed successfully.")
        return X_train, X_test, y_train, y_test
    except Exception as e:
        print(f"Error during data preprocessing: {e}")
        raise



In [ ]:
# Convert to PyTorch tensors
def convert_to_tensors(X_train, X_test, y_train, y_test):
    try:
        print("Converting training and testing data to PyTorch tensors...")
        
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)

        # Debug statements to display tensor shapes
        print(f"X_train_tensor shape: {X_train_tensor.shape}")
        print(f"X_test_tensor shape: {X_test_tensor.shape}")
        print(f"y_train_tensor shape: {y_train_tensor.shape}")
        print(f"y_test_tensor shape: {y_test_tensor.shape}")
        
        print("Conversion to PyTorch tensors completed successfully.")
        
        return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor
    
    except Exception as e:
        print(f"Error converting data to tensors: {e}")
        raise


In [ ]:
import subprocess
import sys
import os
import sqlite3
import json
import logging
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Configure logging for console output
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(console_handler)


# Define DatabaseManager with better connection handling
class DatabaseManager:
    def __init__(self, db_path='training_pipeline.db'):
        self.db_path = db_path
        logger.info(f"Connecting to database at {self.db_path}")
        self.conn = sqlite3.connect(db_path, timeout=5.0, check_same_thread=False)
        self.initialize_database()

    def initialize_database(self):
        try:
            cursor = self.conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS model_config (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    input_dim INTEGER,
                    hidden_dim INTEGER,
                    output_dim INTEGER,
                    dropout_prob REAL,
                    created_at TEXT NOT NULL
                )
            ''')
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS tensor_activations (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    epoch INTEGER,
                    layer TEXT,
                    activations TEXT,
                    created_at TEXT NOT NULL
                )
            ''')
            self.conn.commit()
            logger.info("Database schema initialized.")
        except sqlite3.Error as e:
            logger.error(f"Error initializing database: {e}")
            print(f"Error initializing database: {e}")

    def store_model_config(self, input_dim, hidden_dim, output_dim, dropout_prob):
        try:
            with self.conn:
                cursor = self.conn.cursor()
                cursor.execute('''
                    INSERT INTO model_config (input_dim, hidden_dim, output_dim, dropout_prob, created_at) 
                    VALUES (?, ?, ?, ?, ?)
                ''', (input_dim, hidden_dim, output_dim, dropout_prob, datetime.now().isoformat()))
            logger.info("Model configuration stored.")
        except sqlite3.IntegrityError as e:
            logger.error(f"IntegrityError: {e}")
            print(f"IntegrityError: {e}")

    def store_tensor_activations(self, epoch, layer, activations):
        try:
            with self.conn:
                cursor = self.conn.cursor()
                cursor.execute('''
                    INSERT INTO tensor_activations (epoch, layer, activations, created_at) 
                    VALUES (?, ?, ?, ?)
                ''', (epoch, layer, json.dumps(activations), datetime.now().isoformat()))
            logger.info(f"Stored activations for epoch {epoch} and layer {layer}.")
        except sqlite3.OperationalError as e:
            logger.error(f"Error storing tensor activations: {e}")
            print(f"Error storing tensor activations: {e}")

    def close(self):
        try:
            self.conn.close()
            logger.info(f"Closed database connection to {self.db_path}.")
        except sqlite3.Error as e:
            logger.error(f"Error closing database connection: {e}")
            print(f"Error closing database connection: {e}")


# Function to create DataLoader objects
def create_data_loaders(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, batch_size=32):
    try:
        print("Creating DataLoader objects...")
        
        # Create TensorDataset objects
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

        # Create DataLoader objects
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Debug statements to display the number of batches
        print(f"Number of batches in training DataLoader: {len(train_loader)}")
        print(f"Number of batches in testing DataLoader: {len(test_loader)}")
        
        print("DataLoader objects created successfully.")
        
        return train_loader, test_loader
    
    except Exception as e:
        print(f"Error creating DataLoader objects: {e}")
        raise

# Call the function to create DataLoader objects
X, y = generate_synthetic_data(samples=1000, features=20, classes=2)
X_train, X_test, y_train, y_test = preprocess_data(X, y, test_size=0.2)
X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = convert_to_tensors(X_train, X_test, y_train, y_test)
train_loader, test_loader = create_data_loaders(X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

# Initialize or load the database
def initialize_or_load_db(db_path='training_pipeline.db'):
    try:
        print(f"Attempting to connect to the database at {db_path}...")
        db_manager = DatabaseManager(db_path=db_path)
        print("Database connection established.")
        return db_manager
    except Exception as e:
        print(f"Error initializing or loading the database: {e}")
        raise

# Call the function to initialize or load the database
db_manager = initialize_or_load_db()



In [ ]:
# Model Definition
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.3, db_manager=None):
        super(DNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, hidden_dim)
        self.layer4 = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

        # Store model configuration in the database
        if db_manager:
            db_manager.store_model_config(input_dim, hidden_dim, output_dim, dropout_prob)

        # Visualize initial weights
        self.visualize_weights()

    def visualize_weights(self):
        """Visualizes the weights of each layer."""
        print("Visualizing initial weights...")
        plt.figure(figsize=(12, 8))
        layers = [self.layer1, self.layer2, self.layer3, self.layer4, self.output_layer]
        for i, layer in enumerate(layers):
            weights = layer.weight.data.cpu().numpy()
            plt.subplot(3, 2, i + 1)
            plt.hist(weights.flatten(), bins=30, alpha=0.7, color='blue')
            plt.title(f'Layer {i + 1} Weights Distribution')
            plt.xlabel('Weight Value')
            plt.ylabel('Frequency')
            plt.grid()

        plt.tight_layout()
        plt.show()
        print("Initial weights visualized for all layers.")

    def forward(self, x, epoch=None, db_manager=None):
        print(f"Forward pass input shape: {x.shape}")
        
        x = self.layer1(x)
        print(f"After Layer 1: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer2(x)
        print(f"After Layer 2: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer3(x)
        print(f"After Layer 3: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.layer4(x)
        print(f"After Layer 4: {x.shape}")
        x = self.relu(x)
        x = self.dropout(x)

        x = self.output_layer(x)
        print(f"After Output Layer: {x.shape}")

        # Store activations in the database
        if db_manager and epoch is not None:
            db_manager.store_tensor_activations(epoch, "Output Layer", x.detach().cpu().numpy().tolist())

        return x

# Model instantiation and training parameters
input_dim = X_train.shape[1]  # Set input dimension based on the training data
hidden_dim = 128  # Define hidden layer dimension
output_dim = 2  # Set output dimension for binary classification
dropout_prob = 0.3  # Define dropout probability

# Initialize the DNN model
print("Initializing the DNN model...")
model = DNN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_prob=dropout_prob, db_manager=db_manager)

# Set the optimizer
print("Setting up the optimizer...")
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set the loss function
print("Setting up the loss function (CrossEntropyLoss)...")
criterion = nn.CrossEntropyLoss()


In [ ]:
# Function to plot the training results
def plot_training_results(train_losses, train_accuracies, epochs):
    """
    Plots training loss and accuracy over epochs.
    
    Args:
    - train_losses (list): List of training losses.
    - train_accuracies (list): List of training accuracies.
    - epochs (int): Number of epochs.
    """
    print("Plotting training results...")
    plt.figure(figsize=(12, 5))

    # Plot training loss
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), train_losses, label='Training Loss', color='blue')
    plt.title('Training Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.xticks(range(1, epochs + 1))
    plt.legend()
    plt.grid()

    # Plot training accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), train_accuracies, label='Training Accuracy', color='green')
    plt.title('Training Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.xticks(range(1, epochs + 1))
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()
    print("Training results plotted successfully.")


In [ ]:
# Function to train the model
def train_model(model, data_loader, optimizer, criterion, epochs=10, device='cpu', db_manager=None):
    model.to(device)
    model.train()

    train_losses = []
    train_accuracies = []

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        print(f"Starting epoch {epoch + 1}/{epochs}...")

        for batch_idx, (inputs, labels) in enumerate(data_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs, epoch=epoch, db_manager=db_manager)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(data_loader)
        accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(accuracy)

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    plot_training_results(train_losses, train_accuracies, epochs)  # Plot results after training

# Train the model
train_model(model, train_loader, optimizer, criterion, epochs=10, db_manager=db_manager)


In [ ]:
def plot_3d_training_results(all_train_losses, all_train_accuracies, num_models):
    if not all_train_losses or not all_train_accuracies or len(all_train_losses) != num_models or len(all_train_accuracies) != num_models:
        print("Error during 3D plotting: Data structure is not valid.")
        return

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    for model_id in range(num_models):
        ax.plot(range(len(all_train_losses[model_id])), all_train_losses[model_id], zs=model_id, zdir='y', label=f'Model {model_id + 1}', alpha=0.5)
        ax.plot(range(len(all_train_accuracies[model_id])), all_train_accuracies[model_id], zs=model_id, zdir='x', label=f'Model {model_id + 1}', alpha=0.5)

    ax.set_xlabel('Epochs')
    ax.set_ylabel('Model ID')
    ax.set_zlabel('Loss/Accuracy')
    ax.set_title('3D Training Loss and Accuracy of Models')
    ax.legend()
    plt.show()


In [ ]:
# Function to plot the training results
def plot_training_results(train_losses, train_accuracies, epochs):
    """
    Plots training loss and accuracy over epochs.
    
    Args:
    - train_losses (list): List of training losses.
    - train_accuracies (list): List of training accuracies.
    - epochs (int): Number of epochs.
    """
    print("Plotting training results...")
    plt.figure(figsize=(12, 5))

    # Plot training loss
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), train_losses, label='Training Loss', color='blue')
    plt.title('Training Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.xticks(range(1, epochs + 1))
    plt.legend()
    plt.grid()

    # Plot training accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), train_accuracies, label='Training Accuracy', color='green')
    plt.title('Training Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.xticks(range(1, epochs + 1))
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()
    print("Training results plotted successfully.")


In [ ]:
# Function to train the model
def train_model(model, data_loader, optimizer, criterion, epochs=10, device='cpu', db_manager=None):
    model.to(device)
    model.train()

    train_losses = []
    train_accuracies = []

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        print(f"Starting epoch {epoch + 1}/{epochs}...")

        for batch_idx, (inputs, labels) in enumerate(data_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs, epoch=epoch, db_manager=db_manager)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(data_loader)
        accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(accuracy)

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    plot_training_results(train_losses, train_accuracies, epochs)  # Plot results after training

# Train the model
train_model(model, train_loader, optimizer, criterion, epochs=10, db_manager=db_manager)


In [ ]:
# Generate output grid predictions using the trained models on the test inputs
try:
    print("Generating output grid predictions...")
    
    # Prepare test inputs
    test_inputs = []
    for id_val in train_ids:
        padded_input = pad_to_30x30(train_challenges[id_val]["test"][0]["input"])
        test_inputs.append(padded_input)
        print(f"Padded input grid for ID {id_val} created with shape: {padded_input.shape}")

    # Initialize a list to store output grids for each model
    output_grids = []
    models = []
    
    # Iterate through each model and generate predictions
    for i, model in enumerate(models):
        try:
            # Get the model prediction for each test input
            predictions = [model_pred(id_val, model).cpu().numpy() for id_val in train_ids]
            output_grids.append(predictions)
            print(f"Model {i + 1} predictions generated successfully.")
        except Exception as e:
            print(f"Error generating predictions for model {i + 1}: {e}")
    
    print("Output grid predictions generation completed.")
except Exception as e:
    print(f"Error during output grid predictions generation: {e}")


In [ ]:
# Visualize the output grids
try:
    for idx, model_outputs in enumerate(output_grids):
        for output_grid in model_outputs:
            try:
                plt.figure(figsize=(6, 6))
                plt.title(f"Predicted Output Grid for Test ID {train_ids[idx]}")
                plt.imshow(output_grid, cmap=_cmap, norm=_norm)
                plt.colorbar()
                plt.show()
                print(f"Output grid for Test ID {train_ids[idx]} visualized successfully.")
            except Exception as e:
                print(f"Error visualizing output grid for Test ID {train_ids[idx]}: {e}")
except Exception as e:
    print(f"Error during output grid visualization: {e}")
